In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv("LabelledData (1).txt",sep=",,,",header=None ,names=['question','type'])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [3]:
df.dropna()

,question,type
0,how did serfdom develop in and then leave russ...,unknown
1,what films featured the character popeye doyle ?,what
2,how can i find a list of celebrities ' real na...,unknown
3,what fowl grabs the spotlight after the chines...,what
4,what is the full form of .com ?,what
5,what contemptible scoundrel stole the cork fro...,what
6,what team did baseball 's st. louis browns bec...,what
7,what is the oldest profession ?,what
8,what are liver enzymes ?,what
9,name the scar-faced bounty hunter of the old w...,unknown


In [4]:
#Data Cleaning

df['type']=df['type'].str.strip()
#df['question'] = df['question'].apply(lambda x: x.encode('ascii','ignore'))

df['question'] = df['question'].apply(lambda x: x.lower())
df['question'] = df['question'].apply(lambda x: x.replace(":",' : '))
df['question'] = df['question'].apply(lambda x: x.replace("!",'. '))
df['question'] = df['question'].apply(lambda x: re.sub('[^a-z0-9\s]','',x))
df['question'] = df['question'].apply(lambda x: "".join(x.strip()))
# df.append([df[df.type=='when']]*5,ignore_index=True)
# df.append([df[df.type=='affirmation']]*5,ignore_index=True)
# df.append([df[df.type=='unknown']]*2,ignore_index=True)

In [5]:
#Checking skewedness of dataset
print df.groupby(df['type']).nunique()

             question  type
type                       
affirmation       102     1
unknown           271     1
what              606     1
when               96     1
who               401     1


In [6]:

from nltk.stem import SnowballStemmer
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [7]:
#Convert a collection of text documents to a matrix of token counts
stemmer = SnowballStemmer('english').stem
def stemmed_tokenizer(text):
    return [stemmer(i) for i in word_tokenize(text)]

In [8]:
# CountVectorizer:Convert a collection of text documents to a matrix of token counts
# In order to limit the number of features we use stemmer

#vectorizer = CountVectorizer(analyzer='word',tokenizer=stemmed_tokenizer,stop_words='english',decode_error='replace')
#('Accuracy :', 0.5387205387205387)

vectorizer = CountVectorizer(analyzer='word',tokenizer=stemmed_tokenizer,decode_error='strict')
#('Accuracy :', 0.9023569023569024)

X = vectorizer.fit_transform(df.question.values)
x_train, x_test, y_train, y_test = train_test_split(X, df['type'], test_size = 0.2)

In [9]:
print x_train.shape
print y_train.shape
print x_test.shape
print y_test.shape

(1186, 3220)
(1186,)
(297, 3220)
(297,)


In [10]:
model = MultinomialNB()
model.fit(x_train,y_train)
# evaluate the model of test data
predictions = model.predict(x_test)
print(classification_report(predictions,y_test))
print("Accuracy :",model.score(x_test,y_test))

             precision    recall  f1-score   support

affirmation       0.89      1.00      0.94        16
    unknown       0.80      1.00      0.89        39
       what       1.00      0.84      0.91       144
       when       0.18      0.80      0.30         5
        who       0.99      0.92      0.96        93

avg / total       0.95      0.90      0.91       297

('Accuracy :', 0.8956228956228957)


In [11]:
given_example=vectorizer.transform(["What time does the train leave"])
model.predict(given_example)

array(['what'], dtype='|S11')

In [12]:
print df.groupby(df['type']).nunique()

             question  type
type                       
affirmation       102     1
unknown           271     1
what              606     1
when               96     1
who               401     1


In [13]:
'''
#Interpretations from above example
-> Dataset is highly skewed. Hence we cannot depend upon final test accuracy alone.
-> high precision value in classification incase of 'who' and 'what' is due to more number of examples.
-> This classifier assumes that your features are independent of one another, which is not the case.
-> Hence the model failed to predict 'What time does the train leave' example!
'''

"\n#Interpretations from above example\n-> Dataset is highly skewed. Hence we cannot depend upon final test accuracy alone.\n-> high precision value in classification incase of 'who' and 'what' is due to more number of examples.\n-> This classifier assumes that your features are independent of one another, which is not the case.\n-> Hence the model failed to predict 'What time does the train leave' example!\n"